# Imports

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

## Image Generator configuration

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
        'pokemones/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 16 images belonging to 3 classes.


In [7]:
validation_generator = test_datagen.flow_from_directory(
        'pokemones/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 0 images belonging to 0 classes.


## Model construction

In [1]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

In [27]:
K.image_data_format()

'channels_last'

Channels_last mean "Height, Width, Channels or Depth"

In [30]:
# Conv -> Relu -> Pool
model = Sequential()
model.add(
    Conv2D(32, (3,3), padding="same",
          input_shape=(150, 150,3))
)
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1)) # Import BatchNormalization as Keras layer
model.add(MaxPooling2D(pool_size =(3,3)))
model.add(Dropout(0.25))


In [32]:
# (Conv -> Relu ) * 2 -> Pool
model.add(
    Conv2D(64, (3,3), padding="same" )
)
model.add(Activation("relu"))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [35]:
# First (and only) set of FC -> RELU layers
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#softmax classifier
model.add(Dense(3))
model.add(Activation("softmax"))